<a href="https://colab.research.google.com/github/raz0208/Natural-Language-Processing-Practices/blob/main/TopicModelling/EmbeddingsAnalysis_TopicModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Topic Modelling

## Semantic Signal Separation

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [38]:
# Read and load dataset
dataset = pd.read_csv('gdb_dataset.csv')

# Show the datasets
### Abstract Embeddings Sample Dataset
print('Node Content:', dataset.shape)
print(dataset)

Node Content: (1000, 4)
     Unnamed: 0                                              title  \
0             0  Phenotypic variability of Niemann-Pick disease...   
1             1  Recurrent hypoglycemia secondary to metformin ...   
2             2  Adaptation of the Ambulatory and Home Care Rec...   
3             3  Multidimensional family therapy in adolescents...   
4             4  Balanced crystalloids versus isotonic saline i...   
..          ...                                                ...   
995         995  Molecular Sex Identification in Dioecious  Hip...   
996         996  Antimicrobial Peptides: Powerful Biorecognitio...   
997         997  Analysis of Metabolites in White Flowers of  M...   
998         998  Improved Cold Tolerance of Mango Fruit with En...   
999         999  Characterization of the Complete Chloroplast G...   

                                              abstract  year  
0    Background Niemann-Pick disease type C (NPC) i...  2018  
1    Back

In [48]:
# Extract only the 'abstract' column and drop others
abstracts = dataset['abstract'].dropna().reset_index(drop=True)

# sample the dataset
abstracts = abstracts.loc[:10]

# Display a few samples to verify
abstracts

,abstract
0,Background Niemann-Pick disease type C (NPC) i...
1,Background Metformin toxicity is well known to...
2,Background Measuring service use and costs is ...
3,Background Substance use and delinquency are c...
4,Objectives Intravenous fluids are one of the m...
5,Background Integrase strand transfer inhibitor...
6,Background Ward rounds are an important and ub...
7,Background Major Trauma remains a leading caus...
8,Background The early prediction of acute kidne...
9,Background Interleukin-33 (IL-33) participates...


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
encoder = SentenceTransformer('paraphrase-MiniLM-L12-v2')
embeddings = encoder.encode(abstracts, show_progress_bar=True)

In [ ]:
embeddings

In [40]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel

In [41]:
# Load ModernBERT tokenizer and model from Hugging Face
model_name = 'answerdotai/ModernBERT-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

In [47]:
# Function to get inpout text and return full text embedding (Edit code to get embedding sentence by sentence)
def get_text_embeddings(text):
    # Tokenize input text
    inputs = tokenizer(text.tolist(), return_tensors='pt', padding=True, truncation=True)

    # Forward pass to get hidden states
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the embeddings (use CLS token for sentence-level embedding)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # shape: [batch_size, hidden_size]

    return cls_embedding.squeeze().numpy()

In [46]:
# Ensure all text entries are strings (handle potential non-string entries)
abstract_cleaned = abstracts.astype(str)

# Get embeddings for each abstract
full_text_embeddings = get_text_embeddings(abstract_cleaned)

# Show results
print("\nEmbeddings shape:", full_text_embeddings.shape)
print("First embedding vector (10 values):", full_text_embeddings[0][:10])

AttributeError: 'str' object has no attribute 'astype'